In [ ]:
import numpy as np
from scipy.stats import norm

from randomgen import Generator, Xoshiro512

from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

import math
import csv
from pathlib import Path
from time import time
import os


In [ ]:
def priceBS(S, K, sigma, r, q, T):
    ''' The Black?~@~SScholes formula calculates the price of European put option.
    
    Parameters
    ==========
    S_0    : (float) initial value of the stock prices.
    K     : (float) the strike or exercise price.
    sigma : (float) volatility.
    r     : (float) risk-free interest rate.
    q     : (float) dividend yield rate. 
    N     : (int) Number of paths generated.      
    
    Returns
    =======
    Peu : float
        the price of European put option
    '''

    a = np.log(S/K); b = r - q + 0.5*sigma**2; c = sigma*np.sqrt(T);
    d1 = (a + b*T)/c;
    d2 = d1 - c;

    Peu = - S*np.exp(-q*T)*norm.cdf(-d1) + K*np.exp(-r*T)*norm.cdf(-d2)
    Delta = -np.exp(-q*T)*norm.cdf(-d1)

    return Peu, Delta

In [ ]:
def generate_paths(S_0, dt , sigma, r, N, m,q):
    ''' Function to generte stock paths.
    
    Parameters
    ==========
    S_0    : (float) initial value of the stock prices.
    K     : (float) the strike or exercise price.
    sigma : (float) volatility.
    r     : (float) risk-free interest rate.
    q     : (float) dividend yield rate. 
    N     : (int) Number of paths generated.      
    m     : (int) number of time steps.
    dt    : (float) time step discretization.
    
    Returns
    =======
    S : matrix generated paths
    '''

    seed = 123
    rand = Generator(Xoshiro512())

    S = np.zeros((m + 1, N))
    S[0] = S_0
    for t in range(1, m + 1):
        S[t] = S[t - 1] * np.exp((r - q - sigma ** 2 / 2) * dt + sigma * dt ** 0.5 * rand.standard_normal(N))
    return S

In [ ]:
#train
def pricing_option_Train(K,r,sigma,N,m,S_0,T,q):

    # Time step
    dt = T/m
    # Discount factor
    df = math.exp(-r*dt)
    # Creation of a stock path matrix
    S = generate_paths(S_0, dt , sigma, r, N, m,q);
    # Delta
    Delta = math.exp(-r*T)*S[-1]/S_0*(K>S[-1])

    # Creation of the exercise price matrix
    h = np.maximum(K-S,0)

    PE = np.zeros((m + 1, N))
    DE = np.zeros((m + 1, N))
    for t in range (0,m):
        PE[t], DE[t] = priceBS(S[t], K, sigma, r, q, T-t*dt)
    PE[m] = np.maximum(K-S[m], 0)

    # Actualize the prices
    avg = np.zeros(m+1)
    for t in range (0,m+1):
        avg[t] = np.mean(PE[t])

    #---------------  "Pam - Bally"  ------------------------    
    V = np.zeros(N); # see Bally's suggestion 

    #Iteration over the paths
    ErrCV = [];
    for t in range (m-1,0,-1):

        # we will use as a control variable the price of the associated European
        Peu, Deu = priceBS(S[t], K, sigma, r, q, T-t*dt)
        PHI = np.maximum(K-S[t],0); OBS = PHI - Peu

        # We can stick to the more traditional test to consider all the in-the-money situations
        index = np.nonzero(PHI>0);
#        index = np.nonzero(OBS>0);
        X = S[t][index]; X=X[:,None]; Y = df*V[index];
        if len(X)==0:
            V[index] = df*V[index]
            continue

        rn = MLPRegressor(hidden_layer_sizes=Layers_PamB,max_iter=200,random_state=1,learning_rate_init=0.001,alpha=0.0001)
        Model_PamB[t] = rn.fit(X,Y); c = rn.predict(X)

        registre = np.nonzero(OBS[index]>c);
        sol = index[0][registre[0]]
        dif = np.setdiff1d(range(N),sol)

        for i in range(len(sol)): V[sol[i]] = OBS[sol[i]]
        for i in range(len(dif)): V[dif[i]] = df*V[dif[i]]

    V0= np.mean(V);
    #del Peu, Deu
    Peu, Deu = priceBS(S_0, K, sigma, r, q, T)
    PHI = np.maximum(K-S_0,0); OBS = PHI - Peu
    PamB = np.maximum(OBS,df*V) + Peu
    #DamCV = np.mean(Delta) + Deu

    del V, V0, Peu

        #---------------  "Pam - LS"  ------------------------    
    #price of the option at time T = Initialization
    V=np.copy(h[m]);

    #Iteration over the paths
    for t in range (m-1,0,-1):

        index = np.nonzero(h[t]>0);
        X = S[t][index]; X=X[:,None]; Y = df*V[index];
        if len(X)==0:
            V[index] = df*V[index]
            continue

        rn = MLPRegressor(hidden_layer_sizes=Layers_Pam,max_iter=200,random_state=1,learning_rate_init=0.001,alpha=0.0001)
        Model_Pam[t] = rn.fit(X,Y); c = rn.predict(X)

        # registre = np.nonzero(h[t][index]>np.maximum(c,PE[t][index])); # FABIAN
        registre = np.nonzero(h[t][index]>c); # RAUL

        sol = index[0][registre[0]]
        dif = np.setdiff1d(range(N),sol)
        for i in range(len(sol)): V[sol[i]] = h[t][sol[i]]
        for i in range(len(dif)): V[dif[i]] = df*V[dif[i]]

#    V0= np.mean(V);   # This ignores the deep-in-the-money situations

    # Pam = np.maximum(np.maximum(K-S_0,0), df*V); #FABIAN
    Pam = df*V #RAUL 
    Dam = np.mean(Delta)

    del V

    #---------------  "Pam - Rasmussen"  ------------------------    
    #price of the option at time T = Initialization
    V=np.copy(h[m]);

    exercice_times = -np.ones(m+1)

    #Iteration over the paths
    ErrCV = [];
#    EE = np.copy(h[m])   # EE in the thesis in nothing else than V
    EEE = np.copy(h[m])
    for t in range (m-1,0,-1):

#        EE = df*EE
        EEE = df*EEE
        index = np.nonzero(h[t]>0);
        X = S[t][index]; X=X[:,None];
        V = df*V;  # we update all the positions!
        Y = V[index];

        if len(X)==0: continue

        rn = MLPRegressor(hidden_layer_sizes=Layers_PamR,max_iter=200,random_state=1,learning_rate_init=0.001,alpha=0.0001)
        Model_PamR[t] = rn.fit(X,Y); c = rn.predict(X)

        rn = MLPRegressor(hidden_layer_sizes=Layers_PamR,max_iter=200,random_state=1,learning_rate_init=0.001,alpha=0.0001)
        Model_PamR_pEEE[t] = rn.fit(X,EEE[index]); cEEE = rn.predict(X)

        rn = MLPRegressor(hidden_layer_sizes=Layers_PamR,max_iter=200,random_state=1,learning_rate_init=0.001,alpha=0.0001)
        Model_PamR_pEEE2[t] = rn.fit(X,EEE[index]*EEE[index]); cEEE2 = rn.predict(X)

        rn = MLPRegressor(hidden_layer_sizes=Layers_PamR,max_iter=200,random_state=1,learning_rate_init=0.001,alpha=0.0001)
        Model_PamR_pVEEE[t] = rn.fit(X,V[index]*EEE[index]); cVEEE = rn.predict(X)

#        print("Corr: ", np.corrcoef(c,cEEE))
#        print(np.mean(cEEE-PE[t][index]))
        correction = cEEE-PE[t][index]

        theta = (np.mean(cVEEE)-np.mean(c)*np.mean(cEEE))/(np.mean(cEEE2)-np.mean(cEEE)**2)
#        print("theta = ", theta)
#        theta = 1  # theta has to be properly computed

        registre = np.nonzero(h[t][index]>np.maximum(c-theta*correction,PE[t][index]));
        sol = index[0][registre[0]]

#        dif = np.setdiff1d(range(N),sol)    # no longer useful
#        for i in range(len(sol)): V[sol[i]] = h[t][sol[i]]

        for i in range(len(sol)):
            V[sol[i]] = np.copy(h[t][sol[i]])
#            EE[sol[i]] = np.copy(h[t][sol[i]])
            EEE[sol[i]] = np.copy((PE[t])[sol[i]])

    theta = np.cov(V,EEE)[0][1]/(np.mean(EEE*EEE)-np.mean(EEE)**2)
    V0= np.maximum(np.maximum(K-S_0,0), df*V-theta*(df*EEE-PE[0]));
    PamR = V0
    Dam = np.mean(Delta)

    del V, V0

    return Model_PamB, Model_Pam, Model_PamR, Model_PamR_pEEE, Model_PamR_pEEE2, Model_PamR_pVEEE        

In [ ]:
#test
def pricing_option(K,r,sigma,N,m,S_0,T,q,Model_PamB,Model_Pam,Model_PamR,Model_PamR_pEEE,Model_PamR_pEEE2,Model_PamR_pVEEE):

    # Time step
    dt = T/m
    # Discount factor
    df = math.exp(-r*dt)
    # Creation of a stock path matrix
    S = generate_paths(S_0, dt , sigma, r, N, m,q);
    # Delta
    Delta = math.exp(-r*T)*S[-1]/S_0*(K>S[-1])

    # Creation of the exercise price matrix
    h = np.maximum(K-S,0)

    PE = np.zeros((m + 1, N))
    DE = np.zeros((m + 1, N))
    for t in range (0,m):
        PE[t], DE[t] = priceBS(S[t], K, sigma, r, q, T-t*dt)
    PE[m] = np.maximum(K-S[m], 0)

    # Actualize the prices
    avg = np.zeros(m+1)
    for t in range (0,m+1):
        avg[t] = np.mean(PE[t])

    #---------------  "Pam - Bally"  ------------------------    
    V = np.zeros(N); # see Bally's suggestion 

    #Iteration over the paths
    ErrCV = [];
    for t in range (m-1,0,-1):

        # we will use as a control variable the price of the associated European
        Peu, Deu = priceBS(S[t], K, sigma, r, q, T-t*dt)
        PHI = np.maximum(K-S[t],0); OBS = PHI - Peu

        # We can stick to the more traditional test to consider all the in-the-money situations
        index = np.nonzero(PHI>0);
#        index = np.nonzero(OBS>0);
        X = S[t][index]; X=X[:,None]; Y = df*V[index];
        if len(X)==0:
            V[index] = df*V[index]
            continue

        c = Model_PamB[t].predict(X)

        registre = np.nonzero(OBS[index]>c);
        sol = index[0][registre[0]]
        dif = np.setdiff1d(range(N),sol)

        for i in range(len(sol)): V[sol[i]] = OBS[sol[i]]
        for i in range(len(dif)): V[dif[i]] = df*V[dif[i]]

    V0= np.mean(V);
    #del Peu, Deu
    Peu, Deu = priceBS(S_0, K, sigma, r, q, T)
    PHI = np.maximum(K-S_0,0); OBS = PHI - Peu
    PamB = np.maximum(OBS,df*V) + Peu
    #DamCV = np.mean(Delta) + Deu

    del V, V0, Peu

    #---------------  "Pam - LS"  ------------------------    
    #price of the option at time T = Initialization
    V=np.copy(h[m]);

    #Iteration over the paths
    for t in range (m-1,0,-1):

        index = np.nonzero(h[t]>0);
        X = S[t][index]; X=X[:,None]; Y = df*V[index];
        if len(X)==0:
            V[index] = df*V[index]
            continue

        c = Model_Pam[t].predict(X)

        # registre = np.nonzero(h[t][index]>np.maximum(c,PE[t][index])); #FABIAN
        registre = np.nonzero(h[t][index]>c); # RAUL

        sol = index[0][registre[0]]
        dif = np.setdiff1d(range(N),sol)
        for i in range(len(sol)): V[sol[i]] = h[t][sol[i]]
        for i in range(len(dif)): V[dif[i]] = df*V[dif[i]]

#    V0= np.mean(V);   # This ignores the deep-in-the-money situations

    # Pam= np.maximum(np.maximum(K-S_0,0), df*V); #FABIAN
    Pam = df*V #RAUL  
    Dam = np.mean(Delta)

    del V

    #---------------  "Pam - Rasmussen"  ------------------------    
    #price of the option at time T = Initialization
    V=np.copy(h[m]);

    exercice_times = -np.ones(m+1)

    #Iteration over the paths
    ErrCV = [];
#    EE = np.copy(h[m])   # EE in the thesis in nothing else than V
    EEE = np.copy(h[m])
    for t in range (m-1,0,-1):

#        EE = df*EE
        EEE = df*EEE
        index = np.nonzero(h[t]>0);
        X = S[t][index]; X=X[:,None];
        V = df*V;  # we update all the positions!
        Y = V[index];

        if len(X)==0: continue

        c = Model_PamR[t].predict(X)

        cEEE = Model_PamR_pEEE[t].predict(X)

        cEEE2 = Model_PamR_pEEE2[t].predict(X)

        cVEEE = Model_PamR_pVEEE[t].predict(X)

#        print("Corr: ", np.corrcoef(c,cEEE))
#        print(np.mean(cEEE-PE[t][index]))
        correction = cEEE-PE[t][index]

        theta = (np.mean(cVEEE)-np.mean(c)*np.mean(cEEE))/(np.mean(cEEE2)-np.mean(cEEE)**2)
#        print("theta = ", theta)
#        theta = 1  # theta has to be properly computed

        registre = np.nonzero(h[t][index]>np.maximum(c-theta*correction,PE[t][index]));
        sol = index[0][registre[0]]

#        dif = np.setdiff1d(range(N),sol)    # no longer useful
#        for i in range(len(sol)): V[sol[i]] = h[t][sol[i]]

        for i in range(len(sol)):
            V[sol[i]] = np.copy(h[t][sol[i]])
#            EE[sol[i]] = np.copy(h[t][sol[i]])
            EEE[sol[i]] = np.copy((PE[t])[sol[i]])

    theta = np.cov(V,EEE)[0][1]/(np.mean(EEE*EEE)-np.mean(EEE)**2)
    V0= np.maximum(np.maximum(K-S_0,0), df*V-theta*(df*EEE-PE[0]));
    PamR = V0
    Dam = np.mean(Delta)

    del V, V0

    return np.mean(Pam), np.mean(PamB), np.mean(PamR), np.std(Pam), np.std(PamB), np.std(PamR)


In [ ]:
# Bally's exemple
K=100.; S_0=100.; T=1.; r=np.log(1.1); q=0.; sigma=0.20

Layers_Pam =  [25,25,25,25]
Layers_PamB = [35,35,35,35]
Layers_PamR = [50,50,50,50]


Nic=1000;
for m in [10, 20, 50, 100, 200, 500]:
    print("m = ",m)
    Model_PamB=[]; Model_Pam=[]; Model_PamR=[]; Model_PamR_pEEE=[]; Model_PamR_pEEE2=[]; Model_PamR_pVEEE=[]
    for i in range(0,m):
        Model_PamB.append([]), Model_Pam.append([]), Model_PamR.append([]), Model_PamR_pEEE.append([]), Model_PamR_pEEE2.append([]), Model_PamR_pVEEE.append([])

   # if m == 10 : Layers=[10,10]
   # if m == 20 : Layers=[20,20]
   # if m == 50 : Layers=[50,50]
   # if m == 100 : Layers=[25,25,25,25]
   # if m == 200 : Layers=[50,50,50,50]
   # if m == 500 : Layers=[70,70,70,70] 


    N_train = 100000
    Time0 = time()
    Model_PamB, Model_Pam, Model_PamR, Model_PamR_pEEE, Model_PamR_pEEE2, Model_PamR_pVEEE = pricing_option_Train(K,r,sigma,N_train,m,S_0,T,q)
    Time_train  = time() - Time0

    N = 5000
    fname = 'AmOp_put_RN_N'+str(N)+'_m'+str(m)+'_Test.csv'
    nlines = 0
    for i in range (Nic+1):
        if Path(fname).is_file():
            myFile = open(fname,"r",newline="\n")
            nlines = len(myFile.readlines())
            print('nlines = ',nlines)
            myFile.close()
        if nlines >= Nic+1: break
        Time0 = time()
        Pam,PamB,PamR,std_Pam,std_PamB,std_PamR = pricing_option(K,r,sigma,N,m,S_0,T,q,Model_PamB,Model_Pam,Model_PamR,Model_PamR_pEEE,Model_PamR_pEEE2,Model_PamR_pVEEE)
        Time  = time() - Time0
        myFile = open(fname,"a",newline="\n")
        with myFile:
            fw = csv.writer(myFile, quoting=csv.QUOTE_NONNUMERIC)
            fw.writerow([m, N, Pam, PamB, PamR, std_Pam, std_PamB, std_PamR, Time, Time_train])
            myFile.close()
